In [86]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## Load Preprocessed Data

In [87]:
print("Load  files")
%time df = pd.read_csv('data/train_data.csv', index_col=0)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(df),df['is_fraud'].sum()))

Load  files
CPU times: user 14.7 s, sys: 2.51 s, total: 17.2 s
Wall time: 18.3 s
1296675 transactions loaded, containing 7506 fraudulent transactions


## Feature Selection

In [106]:
# df.info()

In [107]:
df.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'street', 'city', 'state', 'zip', 'lat', 'long',
       'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud', 'trans_date', 'gender_M', 'job_category',
       'job_encoded', 'category_encoded', 'age', 'age_group', 'distance',
       'avg_distance_30_days', 'distance_over_avg_30_days',
       'sum_amount_30_days', 'count_amount_30_days', 'avg_amount_30_days',
       'amount_over_average_30_days', 'avg_distance_90_days',
       'distance_over_avg_90_days', 'sum_amount_90_days',
       'count_amount_90_days', 'avg_amount_90_days',
       'amount_over_average_90_days', 'avg_distance_180_days',
       'distance_over_avg_180_days', 'sum_amount_180_days',
       'count_amount_180_days', 'avg_amount_180_days',
       'amount_over_average_180_days', 'transaction_hour',
       'transaction_day_of_week', 'is_holiday', 'is_weekend',
       'daily_trans_count'

In [120]:
drop_col = ['trans_date_trans_time', 'cc_num', 'merchant', 'category', 
'first', 'last', 'street', 'city', 'state', 'zip', 'job', 'dob', 'unix_time', 
'job_category', 'trans_date', 'trans_num', 'is_fraud']

In [121]:
input_feature = [col for col in df.columns if col not in drop_col]

In [ ]:
# X = df.drop(drop_col, axis=1).select_dtypes(include=np.number)
# y = df['is_fraud']

: 

In [ ]:
# X.info()

: 

## Feature Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X, y)
feature_importances = clf.feature_importances_
X.columns[np.argsort(feature_importances)[::-1]]

## Baseline

Naive classiifier (only predicting non-fraud)

since the testset has around 6 months of data while training... 

In [123]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(max_depth = 2, random_state=0)

model_and_predictions_dictionary = fit_model_and_get_predictions(classifier, train_df, val_df, 
                                                                 input_feature, "is_fraud",
                                                                 scale=False)

In [124]:
model_and_predictions_dictionary

{'classifier': DecisionTreeClassifier(max_depth=2, random_state=0),
 'predictions_val': array([0.        , 0.        , 0.        , ..., 0.77986348, 0.        ,
        0.77986348]),
 'predictions_train': array([0.02135678, 0.02135678, 0.02135678, ..., 0.        , 0.        ,
        0.        ]),
 'training_execution_time': 0.2948899269104004,
 'prediction_execution_time': 0.1955709457397461}

In [127]:
val_df['fraud_detected']=model_and_predictions_dictionary['predictions_val']
# val_df.head()

In [135]:
predictions_df=val_df
predictions_df['predictions']=model_and_predictions_dictionary['predictions_val']
    
performance_assessment(predictions_df, top_k_list=[100])

,AUC ROC,Average precision,Card Precision@100
0,1.0,1.0,0.019


In [137]:
predictions_df['predictions']=0.5
    
performance_assessment(predictions_df, top_k_list=[100])

,AUC ROC,Average precision,Card Precision@100
0,0.5,0.011,0.004


### Defining Training and Validation set

In [100]:
train_df, val_df = get_train_val_set(df, "2019-01-01", delta_train=30, delta_delay=7, delta_val=30)

In [59]:
start_date_training = pd.to_datetime("2019-01-01")

In [136]:
val_df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,...,is_high_risk_hour,city_size,merchant_num_trans_1_day_window,merchant_risk_1_day_window,merchant_num_trans_7_day_window,merchant_risk_7_day_window,merchant_num_trans_30_day_window,merchant_risk_30_day_window,fraud_detected,predictions
count,5.242900e+04,52429.000000,52429.000000,52429.000000,52429.000000,5.242900e+04,5.242900e+04,52429.000000,52429.000000,52429.000000,...,52429.000000,52429.000000,52429.000000,52429.000000,52429.000000,52429.000000,52429.000000,52429.000000,52429.000000,52429.000000
mean,4.269839e+17,72.320933,49208.840356,38.507426,-90.289701,9.174423e+04,1.329974e+09,38.511990,-90.290031,0.010509,...,0.341853,0.158328,4.328826,0.003341,28.195464,0.003656,115.853612,0.003781,0.007878,0.007878
std,1.322328e+18,150.992029,26913.831207,4.984322,13.508340,3.101800e+05,7.747542e+05,5.017374,13.518716,0.101976,...,0.659079,0.390938,2.900327,0.034571,11.631319,0.013558,41.781227,0.007567,0.077941,0.077941
min,6.041621e+10,1.000000,1257.000000,20.027100,-156.292000,2.300000e+01,1.328573e+09,19.029798,-157.188386,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.800115e+14,9.670000,26292.000000,34.518900,-96.798000,7.600000e+02,1.329307e+09,34.691846,-96.881863,0.000000,...,0.000000,0.000000,2.000000,0.000000,21.000000,0.000000,91.000000,0.000000,0.000000,0.000000
50%,3.519233e+15,48.070000,48850.000000,39.371600,-87.724600,2.523000e+03,1.330034e+09,39.353806,-87.700475,0.000000,...,0.000000,0.000000,4.000000,0.000000,27.000000,0.000000,118.000000,0.000000,0.000000,0.000000
75%,4.653879e+15,83.890000,72165.000000,41.846700,-80.276000,2.032800e+04,1.330698e+09,41.940289,-80.395801,0.000000,...,0.000000,0.000000,6.000000,0.000000,34.000000,0.000000,140.000000,0.006803,0.000000,0.000000
max,4.992346e+18,11629.340000,99746.000000,65.689900,-67.950300,2.906700e+06,1.331251e+09,66.624674,-66.978732,1.000000,...,2.000000,2.000000,23.000000,1.000000,84.000000,0.250000,285.000000,0.200000,0.779863,0.779863
